In [1]:
# Install required packages
!pip install qiskit
!pip install qiskit-aer
!pip install qiskit-ibm-runtime
!pip install pylatexenc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.0/353.0 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5

In [2]:
# Standard imports
import numpy as np
from qiskit.circuit.library import efficient_su2
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2, Batch
from qiskit_aer.primitives import EstimatorV2

In [3]:
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram, plot_state_city
from qiskit_aer import AerSimulator
qc = QuantumCircuit.from_qasm_file('P5_granite_summit.qasm')
qc.measure_all()

In [4]:
# Connect to IBM Quantum runtime
from qiskit_ibm_runtime import QiskitRuntimeService
# Replace with your actual token
QiskitRuntimeService.save_account(
  token='YOUR_API_KEY',
  channel="ibm_quantum", # `channel` distinguishes between different account types
  overwrite=True
)

In [5]:
# runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# Load service and select least busy real quantum device
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.least_busy(operational=True, simulator=False)

In [6]:
backend

<IBMBackend('ibm_kyiv')>

In [7]:
# First running a basic transpile for backend
from qiskit import transpile
qc_transpiled = transpile(qc, backend, optimization_level=1) #Tried with optimization levels=1,2,3

In [8]:
# Then adjusting optimization levels to 1,2,3 using pass manager
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3) # Preset pass manager with higher optimization

qc_isa = pm.run(qc) #Transpiled quantum circuit on the ibm_kyiv backend

In [9]:
# Run using Qiskit Runtime's Sampler primitive
from qiskit_ibm_runtime import SamplerV2 as Sampler
sampler = Sampler(backend)
qc_job = sampler.run([qc_transpiled], shots=1000) #Tried shot range from (1000 - 1 million)

In [11]:
qc_job.result()

PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=1000, num_bits=44>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2025-04-13 12:47:43', stop='2025-04-13 12:47:45', size=1000>)])}, 'version': 2})

In [16]:
print(max(qc_job.result()[0].data.meas.get_counts().values())) #Max Frequency

1


In [17]:
print(qc_job.result()[0].data.meas.get_counts()) #Full distribution with same values of 1 which does not give any peaked bitstring.

{'11000011010110001110100010001100111001101100': 1, '11011111010010101101010111001010000111110111': 1, '10111010000001001011001100000010110111000111': 1, '11010010001101110100111111110111001110100101': 1, '11000000100101001111010000001000001010111001': 1, '11001101110011101010011100100010100101110011': 1, '11010101100011100010001111110100110001100001': 1, '01011101011010100010100001111111100010011101': 1, '10011001110110001000001001101001110010010110': 1, '01110101010111001000111011010011000100001101': 1, '11100101010110100101001110111000001010000011': 1, '01010110001010011001001001111001110000101001': 1, '11111001011101100111000011010000001000010110': 1, '10010110001001110100101111010001111111101010': 1, '01110001010011101101110000100101101100101010': 1, '11111000101001111000010110110010101001001100': 1, '11111001001011001010101101001101000011001001': 1, '01111000111001111110010010001001010000101110': 1, '11111101000110111000100111110101000101001111': 1, '00010010010000010010100110110